In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import heapq
import os
from functools import total_ordering

# Huffman Coding
## Author: Tsvetan Dimitrov


### Abstract
TODO

###  Lossy vs Lossless Compression
In this section we will look at lossy vs lossless compression and the advantages and disadvantages of both methods. There is no right or wrong method. It all comes down to taking some notice of a number of different factors. 

#### Lossy Compression
The first type is lossy compression which refers to some of the data from the original file being lost after compression is executed. The process is irreversible and once you convert to lossy, you cannot go back. And the more you compress it, the more degradation occurs. JPEG and GIF are both lossy image formats. One of the biggest obvious benefits to using lossy compression is that it results in a significantly reduced file size (smaller than lossless compression method), but it also means there is quality loss. Most tools, plugins, and software out there will let you choose the degree of compression you want to use.

#### Lossless Compression
The other type is lossless compression which refers to compression without any data or quality loss. All of original data can be recovered when the file is uncompressed. RAW, BMP, GIF, and PNG are all lossless image formats. The big benefit to lossless compression is that you can retain the original quality of your data or images and still achieve a smaller file size. This is generally the technique of choice for text or spreadsheet files, where losing words or financial data could pose a problem. In this article we will explore and implement a lossless technique called Huffman Coding. 

### Information Entropy
In information theory, the major goal is for one person (a transmitter) to convey some message (over a channel) to another person (the receiver). To do so, the transmitter sends a series (possibly just one) partial messages that give clues towards the original message. The information content of one of these partial messages is a measure of how much uncertainty this resolves for the receiver. Let us try a simple experiment. We will assume that the weather is equally probable to be at any one of 4 possible states at any given moment. This translates to having the same probability of a state occurring which is 1/4 in our case.

|             | Sunny | Cloudy | Rainy | Foggy |
|-------------|-------|--------|-------|-------|
| probability |  1/4  |  1/4   |  1/4  |  1/4  |

Now the question is what is the minimal number of bits we can use to store each of these states based on their probability? Our probability can be expressed as follows: 
$$\frac{1}{4} = \frac{1}{2^2} = 2^{-2}$$

$$\text{minimum number of bits} = -\log (p) = -\log \frac{1}{4} = 2$$
So we can now add codes to our table for each weather state:

|             | Sunny | Cloudy | Rainy | Foggy |
|-------------|-------|--------|-------|-------|
| probability |  1/4  |  1/4   |  1/4  |  1/4  |
| code        |  00   |  01    |   10  |   11  |

Each code has to uniquely identify the data that it is encoding, otherwise we will not be able to decode it correctly. Let us now try to change the probabilities and calculate their corresponding codes:

|             | Sunny | Cloudy | Rainy | Foggy |
|-------------|-------|--------|-------|-------|
| probability |  1/2  |  1/4   |  1/8  |  1/8  |
| code        |  0    |  10    |  110  |  111  |


The fact that we can derive from our experiment is that the lower the probability value of a data source, the more information a message transfer event has to carry than a data source with a higher probability value. A partial message that cuts the number of possibilities in half transmits one bit of information about the message. In essence, the "information content" can be viewed as how much useful information the message actually contains. The entropy, in this context, is the expected number of bits of information contained in each message, taken over all possibilities for the transmitted message. In information theory entropy is denoted with $H$ and has the following definition:
$$ H(X) = -p\log (p)$$
This gives us the weighted average of bits for the current partial message or state.

### Algorithm overview
Huffman coding is a way to encode information using variable length strings to represent symbols depending on how frequently they appear. The idea is that symbols that are used more frequently should be shorter while symbols that appear more rarely can be longer. This way, the number of bits it takes to encode a given message will be shorter, on average, than if a fixed-length code was used. In messages that include many rare symbols, the string produced by variable-length encoding may be longer than one produced by a fixed-length encoding. The steps that the algorithm executes are the following:

**Step 1**. Take a list of symbols and their probabilities.

**Step 2**. Select two symbols with the lowest probabilities (if multiple symbols have the same probability, select two arbitrarily).

**Step 3**. Create a binary tree out of these two symbols, labeling one branch with a "1" and the other with a "0". It doesn't matter which side you label 1 or 0 as long as the labeling is consistent throughout the problem (e.g. the left side should always be 1 and the right side should always be 0, or the left side should always be 0 and the right side should always be 1).

**Step 4**. Add the probabilities of the two symbols to get the probability of the new subtree.

**Step 5**. Remove the symbols from the list and add the subtree to the list.

**Step 6**. Go back through the list and take the two symbols/subtrees with the smallest probabilities and combine those into a new subtree. Remove the original symbols/subtrees from the list, and add the new subtree to the list.

**Step 7**. Repeat until all of the elements are combined.



### Implementation

Huffman coding uses a greedy algorithm to build a prefix tree that optimizes the encoding scheme so that the most frequently used symbols have the shortest encoding. The prefix tree describing the encoding ensures that the code for any particular symbol is never a prefix of the bit string representing any other symbol. First step is to create a tree node that will carry the needed information and be able to connect to other nodes. We will need the symbol itself, its frequency count and pointers to its direct left and right child:

In [3]:
@total_ordering
class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq
    
    def __eq__(self, other):
        if other == None or not isinstance(other, Node):
            return False
        
        return self.freq == other.freq

<div class="alert alert-block alert-info">
NOTE: Since we have a custom object as our data structure we need to define how two objects should compare by defining `__eq__` for equality and `__lt__` for less than. `@total_ordering` takes care of the rest of the comparison operators by infering the opposites of the already defined ones.
</div>

#### Tree Construction

Now we will need a tree structure, connecting our nodes. In order to determine the binary assignment for a symbol we have to make the leaves of the tree correspond to the symbols and the assignment will be the path it takes to get from the root of the tree to that leaf. The tree construction steps are the following:

**Step 1**. Count the frequency counts of all symbols and store them in a dictionary.

**Step 2**. Construct a new `Node` for each symbol in the dictionary.

**Step 3**. Push each new `Node` into a Min Heap data structure.

**Step 4**. Connect tree nodes by popping the two least frequent symbol nodes from the heap and sum their probabilities into a new parent `Node`.

**Step 5**. Push the newly constructed parent `Node` in the heap.


In [4]:
class HuffmanTree:
    def __init__(self, text):
        self.text = text
        self.heap = []
        self.frequency_dict = {}
        
    def __make_frequency_dict(self):
        for char in self.text:
            if not char in self.frequency_dict:
                self.frequency_dict[char] = 0
            self.frequency_dict[char] += 1
                
    def __make_heap(self):
        for key in self.frequency_dict:
            node = Node(key, self.frequency_dict[key])
            heapq.heappush(self.heap, node)
                    
    def __merge_nodes(self):
        while len(self.heap) > 1:
            left_node = heapq.heappop(self.heap)
            right_node = heapq.heappop(self.heap)

            merged_node = Node(None, left_node.freq + right_node.freq)
            merged_node.left = left_node
            merged_node.right = right_node

            heapq.heappush(self.heap, merged_node)
    
    def construct_tree(self):
        self.__make_frequency_dict()
        self.__make_heap()
        self.__merge_nodes()
        return self.heap

#### Symbol Code Generation

The next step is to generate all the codes for our symbols. We have already constructed our binary tree with summed probabilities. Now we have to traverse it from top to bottom. In our case we choose that all left nodes will add a `0` to their code at each depth level and all right nodes will add a `1`. Once we reach a leaf we store an entry of the form `<symbol>: <code>` in a dictionary and its reverse `<code>: <symbol>` in another dictionary which will be useful for decompression. 

In [5]:
class HuffmanCodeMap:
    def __init__(self, tree):
        self.tree = tree
        self.code_map = {}
        self.reverse_code_map = {}
    
    def __make_codes(self, root, current_code):
        if root == None:
            return
        
        if root.char != None:
            self.code_map[root.char] = current_code
            self.reverse_code_map[current_code] = root.char
            return
        self.__make_codes(root.left, f'{current_code}0')
        self.__make_codes(root.right, f'{current_code}1')
        
    def construct_code_maps(self):
        root = heapq.heappop(self.tree)
        current_code = ''
        
        self.__make_codes(root, current_code)
        return {
            'code_map': self.code_map,
            'reverse_code_map': self.reverse_code_map
        }

#### Bit Padding

We are almost ready to implement our `compress` and `decompress` functions. But we have to bare in mind something about memory which could lead to data corruption if not considered. For example, suppose we want to encode the word **bird**. This could result in the following sequence of thirteen bits:
```
1101001100111
```
When stored on disk the sequence would be padded with 3 additional random bits in order to achieve a count that is a multiple of 8 (which is a power of 2). If those bits were, e.g. `111` we would get:
```
1101001100111111
```
If we were to load this back from disk we will not get the original string but something like **birdk**. To fix this problem, we have to have some way of knowing when we have finished reading back all of the bits that encode our sequence. One way of doing this is to reserve 8 bits at the beginning that will store the number of bits used as padding at the end. Of course we have to remove this padding when decompressing because it is not needed. We do that by reading those first 8 bits, convert them back into a number that shows the count of padding bits and then simply cut that amount of bits off from the end of the bit sequence.

In [10]:
def add_padding(encoded_text):
    extra_padding = 8 - len(encoded_text) % 8
    encoded_text += (extra_padding * '0')

    padded_info = f'{extra_padding:08b}'
    return padded_info + encoded_text

def remove_padding(padded_encoded_text):
    padded_info = padded_encoded_text[:8]
    extra_padding = int(padded_info, 2)

    padded_encoded_text = padded_encoded_text[8:]
    return padded_encoded_text[:-extra_padding]

#### Compression

#### Decompression


In [11]:
class HuffmanCoding:
    def __init__(self, path):
        self.path = path
        self.code_map = {}
        self.reverse_code_map = {}
    
    def __to_bytes(self, padded_encoded_text):
        text_len = len(padded_encoded_text)
        
        if text_len % 8 != 0:
            print("Encoded text not padded properly")
            exit(0)
            
        b = bytearray()
        for i in range(0, text_len, 8):
            byte = padded_encoded_text[i:i + 8]
            b.append(int(byte, 2))
        return b
    
    def __encode_text(self, text):
        encoded_text = ''
        for char in text:
            encoded_text += self.code_map[char]
        return add_padding(encoded_text)
    
    def __decode_text(self, encoded_text):
        current_code = ''
        decoded_text = ''
        
        for bit in encoded_text:
            current_code += bit
            if current_code in self.reverse_code_map:
                char = self.reverse_code_map[current_code]
                decoded_text += char
                current_code = ''
        
        return decoded_text
    
    def compress(self):
        file_name, _ = os.path.splitext(self.path)
        output_path = f'{file_name}.bin'
        
        with open(self.path, 'r+') as input_file, open(output_path, 'wb') as output_file:
            text = input_file.read().rstrip()
            tree = HuffmanTree(text).construct_tree()
            codes = HuffmanCodeMap(tree).construct_code_maps()
            self.code_map = codes['code_map']
            self.reverse_code_map = codes['reverse_code_map']
            
            encoded_text = self.__encode_text(text)
            
            b = self.__to_bytes(encoded_text)
            
            output_file.write(bytes(b))
        
        print(f"File '{self.path}' is compressed as '{output_path}'")
        
        return output_path
    
    def decompress(self, input_path):
        file_name, _ = os.path.splitext(self.path)
        output_path = f'{file_name}_decompressed.txt'
        
        with open(input_path, 'rb') as input_file, open(output_path, 'w') as output_file:
            bit_string = ''
            
            byte = input_file.read(1)
            while byte:
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8, '0')
                bit_string += bits
                byte = input_file.read(1)
                
            encoded_text = remove_padding(bit_string)
            decompressed_text = self.__decode_text(encoded_text)
            
            output_file.write(decompressed_text)
            
        print(f"File '{input_path}' is decompressed as '{output_path}'")
        
        return output_path

            

In [12]:
path = 'sample.txt'
h = HuffmanCoding(path)
output_path = h.compress()
h.decompress(output_path)

def get_file_size(path):
    file_size = os.path.getsize(path)
    return f'{file_size / 1000}KB'

print(get_file_size('sample.bin'))
print(get_file_size('sample_decompressed.txt'))


File 'sample.txt' is compressed as 'sample.bin'
File 'sample.bin' is decompressed as 'sample_decompressed.txt'
394.017KB
715.332KB


### Conclusion

### Future Work

### References